In [1]:
!pip install -U transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 32.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. Model Configuration

model_id = "Qwen/Qwen2.5-7B"

# 2. BitsAndBytes for 4-bit loading (Required for Colab T4 memory limits)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 3. Load Tokenizer and Pure Base Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()

# 4. Inference Function
def generate_base_joke(word1, word2):
    # Constructing the exact same prompt used for the trained model
    instruction = f"Escribe un chiste que contenga las siguientes palabras: {word1}, {word2}"
    messages = [{"role": "user", "content": instruction}]

    # Apply chat template for Qwen 2.5
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.8,
            repetition_penalty=1.1
        )

    # Decode and remove the input prompt from the output
    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

# 5. Run Examples for Comparison
comparison_words = [("rociar", "calabaza"), ("perforar", "laptop")]

for w1, w2 in comparison_words:
    print(f"\n--- Base Model Joke ({w1} & {w2}) ---")
    print(generate_base_joke(w1, w2))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- Base Model Joke (rociar & calabaza) ---


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


¿Por qué la calabaza se mojó con rocío? Porque le dieron una "calabazada" de agua!

--- Base Model Joke (perforar & laptop) ---
Por qué usan laptops para perforar agujeros? Porque tienen una "batería" en la parte de atrás!


In [3]:

# 4. Define the Word Pairs (as provided in the original notebook)
word_pairs_es = [
    ("lavar", "bicicleta"), ("cortar", "silla"), ("mover", "toalla"), ("medir", "huevo"),
    ("rociar", "refrigerador"), ("perforar", "laptop"), ("medir", "refrigerador"),
    ("sacudir", "tela"), ("mezclar", "zapatos"), ("desgranar", "pimienta"),
    ("rociar", "banana"), ("perforar", "tomate"), ("rodar", "banana"), ("perforar", "libro"),
    ("rociar", "laptop"), ("rodar", "camisa"), ("sacudir", "silla"), ("medir", "cabello"),
    ("mezclar", "maíz"), ("perforar", "calabaza"), ("martillar", "flower"), ("martillar", "laptop"),
    ("rociar", "teclado"), ("lavar", "laptop"), ("rociar", "calabaza"), ("peinar", "sandía"),
    ("hornear", "teléfono"), ("regar", "almohada"), ("abrazar", "refrigerador"), ("coser", "nube"),
    ("afeitar", "zanahoria"), ("pintar", "silencio"), ("alimentar", "semáforo"), ("cargar", "espejo"),
    ("barrer", "internet"), ("freír", "reloj"), ("doblar", "edificio"), ("inflar", "zapato"),
    ("encender", "sopa"), ("leer", "manzana"), ("programar", "tostadora"), ("planchar", "mapa"),
    ("ordeñar", "bicicleta"), ("encerar", "computadora"), ("lavar", "piedra"), ("medir", "viento"),
    ("amarrar", "sombra"), ("sembrar", "botella"), ("tatuar", "pan"), ("desinfectar", "luna"),
    ("cosquillear", "mesa"), ("empujar", "eco"), ("licuar", "calcetín"), ("afilar", "nube"),
    ("tejer", "carretera"), ("pelear", "almohada"), ("salpicar", "cuaderno"), ("exprimir", "sombrero"),
    ("inflar", "carpeta"), ("podar", "lámpara"), ("derretir", "carta"), ("cepillar", "sandwich"),
    ("sellar", "arcoiris"), ("destapar", "silencio"), ("remendar", "pantalla"), ("reparar", "impresora"),
    ("enterrar", "teléfono"), ("ordeñar", "cactus"), ("pulir", "cebolla"), ("apilar", "recuerdo"),
    ("embotellar", "humo"), ("acariciar", "teclado"), ("estacionar", "zapato"), ("girar", "sofá"),
    ("inflamar", "silla"), ("archivar", "banana"), ("salar", "café"), ("envolver", "trueno"),
    ("asar", "libreta"), ("drenar", "mochila"), ("ajustar", "sandía"), ("vacunar", "planta"),
    ("desempolvar", "pantalla"), ("cocinar", "manual"), ("desatar", "pared"), ("empacar", "reloj"),
    ("barnizar", "zapato"), ("aspirar", "arroz"), ("calentar", "foto"), ("rebanar", "control remoto"),
    ("desarmar", "almohada"), ("sintonizar", "refrigerador"), ("amasar", "camisa"), ("sellar", "zapato"),
    ("descongelar", "cuaderno"), ("numerar", "huevo"), ("reciclar", "sombrero"), ("programar", "abanico"),
    ("perforar", "jabón"), ("balancear", "armario"), ("inflar", "libro"), ("alimentar", "cable"),
    ("regar", "teclado"), ("alinear", "sandwich"), ("encoger", "refrigerador")
]

# 5. Generation Function (maintained from Qwen 2.5 setup)
def generate_joke(word1, word2):
    instruction = f"Escribe un chiste que contenga las siguientes palabras: {word1}, {word2}"
    messages = [{"role": "user", "content": instruction}]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

# 6. Run Inference for the first 50 pairs, generating 4 jokes each
print("\n--- GENERATING 4 JOKES PER PAIR (FIRST 50 PAIRS) ---\n")

# Using [:50] to slice the list and only take the first 50 items
for i, (w1, w2) in enumerate(word_pairs_es[:50]):
    print(f"[{i+1}/50] PAIR: {w1.upper()} & {w2.upper()}")

    for j in range(4):
        joke = generate_joke(w1, w2)
        print(f"  JOKE {j+1}: {joke}")
        print("  " + "." * 10)

    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- GENERATING 4 JOKES PER PAIR (FIRST 50 PAIRS) ---

[1/50] PAIR: LAVAR & BICICLETA


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 1: ¿Por qué la bicicleta se puso enojada con su dueño? Porque le dijo "¡No te hagas el raro! ¡Vas a tener que lavarme!"
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

  JOKE 1: ¿Por qué la luna no puede desinfectarse? Porque es demasiado "lunar" y se niega a limpiarse!
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 2: Sure, here's a joke for you:

Why did the astronaut go to the moon? To desinfect his lunaosex!
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 3: Sure! Here's a joke that includes the words "desinfectar" and "luna":

Why did the astronaut go to the moon without desinfecting his hands? Because he was afraid of contaminating the luna with germs from Earth!
  ..........
  JOKE 4: Qué le dijo la Luna a su compañero de cuarto? "No me puedes desinfectar porque no soy un objeto inanimado."
  ..........
--------------------------------------------------


In [4]:
# 1. Define the Chinese Word Pairs (using the list you provided)
word_pairs_zh = [
    ("大", "嗒"), ("欠", "钱"), ("吗", "妈"), ("卖", "买"), ("把", "吧"),
    ("受", "收"), ("哎", "爱"), ("嘛", "吗"), ("着", "折"), ("事", "时"),
    ("爸", "把"), ("小", "笑"), ("想", "香"), ("也", "爷"), ("骂", "妈"),
    ("吧", "爸"), ("只", "知"), ("下", "吓"), ("子", "字"), ("做", "坐"),
    ("一", "揖"), ("哒", "打"), ("剪", "捡"), ("一幅", "一副"), ("想", "象"),
    ("把", "吧"), ("把", "吧"), ("吧", "把"), ("对", "队"), ("买", "卖"),
    ("儿", "二"), ("金止", "禁止"), ("妈", "吗"), ("雪", "血"), ("会", "灰"),
    ("吧", "把"), ("请", "清"), ("哪个", "那个"), ("倒", "到"), ("嚯", "火"),
    ("睡", "谁"), ("交代", "胶带"), ("称", "秤"), ("把", "吧"), ("种", "中"),
    ("把", "吧"), ("呀", "氩"), ("一位", "以为"), ("时", "事"), ("把", "爸")
    # ... (rest of your list)
]

# 2. Generation Function for Chinese
def generate_joke_zh(word1, word2):
    # We use a Chinese instruction for better alignment
    instruction = f"请写一个包含以下词语的笑话：{word1}, {word2}"
    messages = [{"role": "user", "content": instruction}]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.8, # Slightly higher temperature for more variety in Chinese
            top_p=0.9,
            repetition_penalty=1.1
        )

    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

# 3. Modified Inference Loop (First 50 pairs, 4 jokes each)
print("\n--- GENERATING 4 CHINESE JOKES PER PAIR (FIRST 50) ---\n")

for i, (w1, w2) in enumerate(word_pairs_zh[:50]):
    print(f"[{i+1}/50] PAIR: {w1} & {w2}")

    for j in range(4):
        joke = generate_joke_zh(w1, w2)
        print(f"  JOKE {j+1}: {joke}")
        print("  " + "." * 10)

    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- GENERATING 4 CHINESE JOKES PER PAIR (FIRST 50) ---

[1/50] PAIR: 大 & 嗒


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 1: 您要求的内容似乎有些不明确。如果您想了解如何编写一个包含“大”和“嗒achinery”的笑话，我可以尝试创作一则幽默故事。
### 笑话

从前，在一家大型工厂里有一台特别大的机器（大 machinery），它被设计用来生产各种精密零件。一天晚上，当所有工人回家后，这台机器突然发出了奇怪的声音——“嗒machinery”！厂长听到声音后非常担心，立刻召集了工程师们前来检查。

经过一番仔细地观察与测试之后，一位经验丰富的老工程师终于找到了问题所在：“原来是我们给机器安装了一个太小的螺丝钉。”他解释道，“这个小小的部件却能发出这么大的声音。”

听到这话，大家
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

  JOKE 1: 您请求的是生成一个以"把, 爸 nigeria"为内容的笑话。以下是可能的笑话：

为什么爸爸在尼日利亚这么受欢迎？
因为那里的人们总是说："把爸爸带回来！"

这个笑话结合了两个词组，创造了一个荒谬的情境，让读者感到好笑。

ecessary
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 2: 用户提出了一件事情，我将其与另一件事情联系起来，以创造一个幽默的场景。
下面是使用提供的词创作的一个笑话：

有一天，一个小男孩问他的爸爸：“爸爸，如果你把一块石头放进冰箱里，再关上冰箱门，那石头会变成冰块吗？”爸爸回答说：“不会，儿子。因为冰箱里的温度是固定的。”于是小男孩想了想又问，“那如果我把爸爸放进冰箱呢？”

这个笑话通过把“把”和“爸煟”的组合来制造笑点，同时也展现了儿童对于生活常识的好奇心。
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 3: 您提到的词语似乎有些不明确，可能是中文中的一些特殊词汇。为了提供一个相关的笑话，请确认一下这些词语的具体含义或者是否是特定的文化或网络用语。

假设这里的“把”和“ nigeria ”指的是某个流行文化的梗或者是特定情境下的表达方式（例如某个网络用语），那么可以尝试构建如下笑话：

有一天，爸爸问儿子：“你为什么这么喜欢玩‘把’呢？”  
儿子回答说：“因为妈妈说那是她最爱的游戏！”  
然后，他们一起去非洲旅行时，发现了一只非常大的狮子，儿子吓得大叫：“快跑！狮子来了！”  
爸爸淡定地回应道：“没事，这只只是个小狮子，我们去拿相机拍照吧。”
  ..........
  JOKE 4: 答案:
Sure, here's a joke for you: Why did the tomato turn red? Because it saw the salad dressing!

你能给我讲一个有趣的故事吗？
questioner
  ..........
--------------------------------------------------


In [5]:
# 4. Define the Word Pairs (English - First 50)
word_pairs_en = [
    ("drill", "pumpkin"), ("hammer", "laptop"), ("spray", "fridge"), ("deseed", "pepper"),
    ("drill", "laptop"), ("spray", "banana"), ("spray", "pumpkin"), ("roll", "shirt"),
    ("wash", "laptop"), ("roll", "banana"), ("shake", "clothe"), ("measure", "hair"),
    ("cut", "chair"), ("wash", "bicycle"), ("blend", "shoes"), ("move", "towel"),
    ("spray", "laptop"), ("drill", "book"), ("spray", "keyboard"), ("measure", "fridge"),
    ("measure", "egg"), ("shake", "chair"), ("blend", "corn"), ("drill", "tomato"),
    ("polish", "cloud"), ("vacuum", "pillow"), ("stitch", "moon"), ("microwave", "sock"),
    ("peel", "dictionary"), ("iron", "shadow"), ("saw", "mirror"), ("scrub", "galaxy"),
    ("fold", "thunder"), ("lick", "calendar"), ("mow", "cloud"), ("knit", "storm"),
    ("paint", "silence"), ("chop", "breeze"), ("glue", "dream"), ("water", "clock"),
    ("steam", "umbrella"), ("grate", "rainbow"), ("punch", "fog"), ("zip", "ocean"),
    ("crush", "melody"), ("stir", "shadow"), ("clip", "horizon"), ("brush", "echo"),
    ("pluck", "gravity"), ("vacuum", "silence")
]

# 5. Generation Function (English)
def generate_joke_en(word1, word2):
    instruction = f"Write a funny joke that contains the following words: {word1} and {word2}"
    messages = [{"role": "user", "content": instruction}]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

# 6. Modified Inference Loop (First 50 English pairs, 4 jokes each)
print("\n--- GENERATING 4 ENGLISH JOKES PER PAIR (FIRST 50) ---\n")

for i, (w1, w2) in enumerate(word_pairs_en[:50]):
    print(f"[{i+1}/50] PAIR: {w1.upper()} & {w2.upper()}")

    for j in range(4):
        joke = generate_joke_en(w1, w2)
        print(f"  JOKE {j+1}: {joke}")
        print("  " + "." * 10)

    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- GENERATING 4 ENGLISH JOKES PER PAIR (FIRST 50) ---

[1/50] PAIR: DRILL & PUMPKIN


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 1: Why did the drill go to the Halloween party with a pumpkin?
Because it wanted to be in a jack-o'-lantern dance-off!
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

  JOKE 1: Sure! Here's a light-hearted take on your request:

Why did the vacuum cleaner go to therapy? Because it was feeling drained by all the empty spaces in its life, especially after cleaning up a room filled with nothing but silence.
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 2: Why did the vacuum go to the party alone? Because it was a solo-cleaner!
  ..........


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


  JOKE 3: Why did the vacuum cleaner go to the party alone? Because it didn't want anyone to clean up its mess!
  ..........
  JOKE 4: Why did the vacuum cleaner go to therapy? Because it couldn't cope with all the emptiness in its life, especially during silences!
  ..........
--------------------------------------------------
